<a href="https://colab.research.google.com/github/geoffcorvera/deeplearning-spring2021/blob/hw4/Grad_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.image import imread
import os

# from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

# Import Model & Data

In [44]:
model = VGG16()

In [53]:
root_dir = r'/content/drive/MyDrive/Spring2021/Deep Learning & Computer Vision/prog4_data'
imgs = []

for i in range(1,6):
  img_path = os.path.join(root_dir, f'gc{i}.jpg')
  imgs.append(imread(img_path))

imgs = list(map(preprocess_input, imgs))

# Get Predictions

In [54]:
prediction = model.predict(imgs)

ValueError: ignored

In [ ]:
fig, axs = plt.subplots(nrows=5, ncols=1)
fig.suptitle('Original Images')

for ax, img in zip(axs.flatten(), imgs):
  ax.imshow(img)
  ax.set_axis_off()

plt.show()